In [246]:
from sklearn.svm import SVC
import numpy as np
import pandas as pd 
import matplotlib.pyplot as plt 
from sklearn import datasets
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.metrics.pairwise import polynomial_kernel
from sklearn.model_selection import cross_val_score
import math
from sklearn.feature_selection import mutual_info_regression, RFE, mutual_info_classif, VarianceThreshold
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score

In [198]:
df = pd.read_csv("LettersTrain.csv")
y_train = df['labels']
x_train = df.iloc[:,1:618]
df_test = pd.read_csv('LettersTest.csv')
x_test = df_test.iloc[:,1:618]
y_test = df_test['labels']

In [31]:
t = [0,100,200,300,400,500,600,700,800,900,1000]
for i in range(10):
    validation = x_train[t[i]:t[i+1]]
    test = x_train[t[i+1]:]
   

In [50]:
model = SVC(kernel='linear',C=math.inf)
score = cross_val_score(model,x_train,y_train,cv=10)
mean = score.mean()
mean

0.9420000000000002

In [9]:
d = pd.DataFrame(columns=['degree','score'])
for i in range(10):
    svc = SVC(kernel='poly',degree=i,C=1e10)
    cv = cross_val_score(svc,x_train,y_train,cv=10)
    m = cv.mean()
    d.loc[i] = [i,m]

In [10]:
d

,degree,score
0,0.0,0.055
1,1.0,0.942
2,2.0,0.939
3,3.0,0.941
4,4.0,0.936
5,5.0,0.930
6,6.0,0.928
7,7.0,0.922
8,8.0,0.917
9,9.0,0.903


In [29]:
import concurrent.futures
li = [i for i in range(1,20)]

def process(p):
    svc = SVC(kernel='poly',degree=p,C=math.inf)
    cv = cross_val_score(svc,x_train,y_train,cv=10)
    m = cv.mean()
    print(m)

with concurrent.futures.ProcessPoolExecutor() as executor:
    executor.map(process,li)

# Question 3

In [182]:
features_corr = x_train.corr()
features = [i for i in x_train.keys()]

In [183]:
f = pd.DataFrame(columns=['f1','f2','corr'])
counter = 1
for i in features_corr:
    for j in features_corr:
        if features_corr[i][j] >= 0.9 and i!=j:
            if (len(i)==3 and len(j)==3 and int(i[-1:]) < int(j[-1:])) or (len(i)==3 and len(j)==4) or (len(i)==4 and len(j)==4 and int(i[-2:]) < int(j[-2:])):
                f.loc[counter] = [i,j,features_corr[i][j]]
                counter +=1
            #print("Correlation of " + str(i) + " and " + str(j) + ": " + str(features_corr[i][j]))

In [184]:
f

,f1,f2,corr
1,F_5,F_37,0.906120
2,F_6,F_7,0.929866
3,F_6,F_38,0.936261
4,F_6,F_39,0.908160
5,F_7,F_8,0.944398
...,...,...,...
61,F_77,F_78,0.926215
62,F_78,F_79,0.928310
63,F_79,F_80,0.912786
64,F_88,F_89,0.903091


In [185]:
for j in f['f2']:
    try:
        del x_train[j]
        features.remove(j)
    except:
        pass

In [186]:
x_train

,F_1,F_2,F_3,F_4,F_5,F_6,F_17,F_18,F_19,F_20,...,F_608,F_609,F_610,F_611,F_612,F_613,F_614,F_615,F_616,F_617
0,-0.4540,0.4474,0.6542,0.3700,-0.3408,-0.3700,0.7028,0.9418,1.0000,0.7382,...,0.1258,0.0994,0.0596,0.1258,0.1390,0.3774,0.1920,-0.1788,-0.2582,-0.4304
1,-0.6208,-0.1356,-0.0166,0.4994,0.5082,0.3760,0.6340,0.6450,0.8676,0.9074,...,0.6908,1.0000,0.3196,0.4226,0.2990,0.2372,0.1958,0.1752,0.1134,-0.6494
2,-0.3194,0.0722,0.2634,0.3986,0.8228,0.8554,0.9814,1.0000,0.7436,0.7156,...,0.7220,0.7968,0.8182,0.8716,0.7754,0.8610,0.7220,0.8074,0.6578,0.5828
3,-0.3814,0.5180,0.3738,0.6888,0.4952,0.5104,-0.6660,-0.6964,-0.5978,-0.5978,...,-0.1892,-0.3334,-0.3514,-0.2612,-0.2072,-0.0810,-0.2792,-0.2792,-0.2612,-0.6756
4,-0.6580,0.0968,0.1678,0.4580,0.6064,0.6258,0.3484,0.4580,0.4774,0.7742,...,0.4270,0.2982,0.5322,0.6608,1.0000,0.5438,0.3334,0.3918,0.4270,0.3334
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
995,-0.6210,-0.1048,0.4112,0.7178,0.6532,0.2096,1.0000,0.8870,0.8226,0.7904,...,0.8158,0.6974,0.6448,0.5658,0.6710,0.8290,0.6974,0.6974,0.8158,0.1184
996,-0.5634,-0.2058,-0.0602,0.4304,0.8046,0.9584,-0.3762,-0.2724,0.1226,0.0520,...,0.7216,0.4178,0.1392,0.2152,0.1646,0.1392,-0.0380,0.4178,0.2152,-0.2912
997,-0.2952,0.1382,0.6064,0.7410,0.9676,0.7534,0.8256,0.7334,0.5816,0.5168,...,1.0000,0.8812,0.5446,0.5050,0.4258,0.3466,0.2674,0.5842,0.3664,-0.2872
998,-0.1266,0.4938,0.4144,0.5286,0.2656,-0.2308,-0.0422,0.2060,0.2754,0.6972,...,0.4256,0.3192,0.8086,0.7660,0.7234,0.4468,0.0638,-0.0426,0.0000,-0.7660


In [187]:
threshold_n = .5
sel = VarianceThreshold(threshold=(threshold_n))
sel.fit_transform(x_train)

array([[-1.    , -1.    , -1.    , ...,  0.419 , -1.    ,  1.    ],
       [-1.    , -1.    , -0.7428, ..., -0.145 , -1.    ,  1.    ],
       [-1.    ,  0.8286,  1.    , ...,  0.2434, -1.    ,  1.    ],
       ...,
       [-1.    , -1.    , -1.    , ..., -0.2946, -1.    ,  1.    ],
       [-1.    , -0.4858, -0.1714, ...,  1.    , -1.    ,  1.    ],
       [-1.    , -1.    , -1.    , ...,  1.    , -1.    ,  1.    ]])

In [188]:
sel

VarianceThreshold(threshold=0.5)

In [189]:
MI = np.round(1000*mutual_info_classif(x_train,y_train)).astype(np.float)/1000

In [190]:
p = [(MI[i],features[i]) for i in range(len(MI)) if MI[i]<0.2]

In [191]:
for i in p:
    features.remove(i[1])
    del x_train[i[1]]

In [192]:
x_train

,F_2,F_3,F_4,F_5,F_6,F_17,F_18,F_19,F_20,F_21,...,F_590,F_591,F_592,F_593,F_610,F_611,F_613,F_615,F_616,F_617
0,0.4474,0.6542,0.3700,-0.3408,-0.3700,0.7028,0.9418,1.0000,0.7382,0.6704,...,-0.2980,-0.3642,-0.4304,-0.2716,0.0596,0.1258,0.3774,-0.1788,-0.2582,-0.4304
1,-0.1356,-0.0166,0.4994,0.5082,0.3760,0.6340,0.6450,0.8676,0.9074,0.8920,...,0.3608,0.1958,0.3814,0.3196,0.3196,0.4226,0.2372,0.1752,0.1134,-0.6494
2,0.0722,0.2634,0.3986,0.8228,0.8554,0.9814,1.0000,0.7436,0.7156,0.8368,...,-0.4332,-0.1550,-0.2086,0.0802,0.8182,0.8716,0.8610,0.8074,0.6578,0.5828
3,0.5180,0.3738,0.6888,0.4952,0.5104,-0.6660,-0.6964,-0.5978,-0.5978,-0.5180,...,-0.2792,-0.1892,-0.0450,-0.0270,-0.3514,-0.2612,-0.0810,-0.2792,-0.2612,-0.6756
4,0.0968,0.1678,0.4580,0.6064,0.6258,0.3484,0.4580,0.4774,0.7742,0.9806,...,-0.4620,-0.5556,-0.3100,-0.2982,0.5322,0.6608,0.5438,0.3918,0.4270,0.3334
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
995,-0.1048,0.4112,0.7178,0.6532,0.2096,1.0000,0.8870,0.8226,0.7904,0.6290,...,-0.2500,-0.2500,0.3422,0.5922,0.6448,0.5658,0.8290,0.6974,0.8158,0.1184
996,-0.2058,-0.0602,0.4304,0.8046,0.9584,-0.3762,-0.2724,0.1226,0.0520,-0.0436,...,0.1646,0.4936,0.7974,1.0000,0.1392,0.2152,0.1392,0.4178,0.2152,-0.2912
997,0.1382,0.6064,0.7410,0.9676,0.7534,0.8256,0.7334,0.5816,0.5168,0.5692,...,0.5842,0.2476,0.5842,0.5050,0.5446,0.5050,0.3466,0.5842,0.3664,-0.2872
998,0.4938,0.4144,0.5286,0.2656,-0.2308,-0.0422,0.2060,0.2754,0.6972,0.6824,...,0.4042,0.4042,0.2978,0.5106,0.8086,0.7660,0.4468,-0.0426,0.0000,-0.7660


In [200]:
scaler = StandardScaler()
x_scaled = scaler.fit_transform(x_train)

In [194]:
for i in ['linear','poly','rbf','sigmoid']:
    sv = SVC(kernel=i)
    c = cross_val_score(sv,x_scaled,y_train,cv=10)
    mean = c.mean()
    print(mean)

0.925
0.768
0.917
0.9110000000000001


In [256]:
sv = SVC(kernel='poly',C=1e10,degree=1)
c = cross_val_score(sv,x_train,y_train,cv=10)
mean = c.mean()
print(i,mean)

0.0013 0.9420000000000002


# Question 4

In [249]:
mod = SVC(kernel='rbf',C=1e10)
cvs = cross_val_score(mod,x_train,y_train,cv=10)
print(i, cvs.mean())

0.0013 0.943


In [250]:
mod.fit(x_train,y_train)
pred = mod.predict(x_test)
acc= accuracy_score(y_test,pred)
acc

0.9083333333333333

In [257]:
sv = SVC(kernel='sigmoid',C=8,gamma=0.002)
sv.fit(x_train,y_train)
pred_poly = sv.predict(x_test)
acc_poly = accuracy_score(y_test,pred_poly)
acc_poly

0.8833333333333333

In [258]:
s = SVC(kernel='poly',C=1e10,degree=1)
s.fit(x_train,y_train)
pred_sig = s.predict(x_test)
acc_sig = accuracy_score(y_test,pred_sig)
acc_sig

0.8833333333333333

# Question 6

In [301]:
import random
co = pd.DataFrame(columns=['acc_test','acc_train','nb_vec','gamma'])
for i in [random.uniform(0.0001,0.9) for j in range(10)]:
    pol = SVC(kernel='rbf',gamma=i)
    pol.fit(x_train,y_train)
    p = pol.predict(x_test)
    p_train = pol.predict(x_train)
    ac_train = accuracy_score(y_train,p_train)
    ac_test = accuracy_score(y_test,p)
    vec = pol.n_support_.sum()
    co.loc[i] = [ac_test,ac_train, vec, i]
    print(f"Gamma: {i}, accuracy test: {ac_test}, accuracy train: {ac_train}")

Gamma: 0.4745780343475704, accuracy test: 0.016666666666666666, accuracy train: 1.0
Gamma: 0.2700714864156522, accuracy test: 0.016666666666666666, accuracy train: 1.0
Gamma: 0.8641846092116927, accuracy test: 0.016666666666666666, accuracy train: 1.0
Gamma: 0.7594560459668204, accuracy test: 0.016666666666666666, accuracy train: 1.0
Gamma: 0.6608686918038793, accuracy test: 0.016666666666666666, accuracy train: 1.0
Gamma: 0.591492810152453, accuracy test: 0.016666666666666666, accuracy train: 1.0
Gamma: 0.8928661810492563, accuracy test: 0.016666666666666666, accuracy train: 1.0
Gamma: 0.8599789686104844, accuracy test: 0.016666666666666666, accuracy train: 1.0
Gamma: 0.6473548426381188, accuracy test: 0.016666666666666666, accuracy train: 1.0
Gamma: 0.3638556181000055, accuracy test: 0.016666666666666666, accuracy train: 1.0


In [268]:
co['acc'].corr(co['nb_vec'])

-0.9830708809285906

In [290]:
co['degree'].corr(co['nb_vec'])

0.6910861353479849

In [288]:
from sklearn.linear_model import SGDClassifier
c = SGDClassifier()
c.fit(x_train,y_train)
pre = c.predict(x_train)
ac = accuracy_score(y_train,pre)
ac

0.999